In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from pathlib import Path
pd.set_option('display.max_rows', 100)


In [2]:
dialogpath = Path('/Users/michelleschlacks/Desktop/movie_dialogue.tsv')
chars = pd.read_csv(dialogpath, sep = '\t')
chars = chars.sample(frac = 1.0)
                  
chars.head()


,mid,cid,cname,mname,gender,wordcount,year,genres,comedy,thriller,drama,romance,lines
2342,m533,u7883,JACKIE,stepmom,f,3037,1998,"['comedy', 'drama']",True,False,True,False,Good. That's very good. / Don't be. I'm goin...
2814,m70,u1070,BONES,get shorty,m,242,1995,"['comedy', 'crime', 'thriller']",True,True,False,False,You say look at you? / What? / Where's Leo Dev...
2102,m489,u7245,HAN,star wars: episode vi - return of the jedi,m,403,1983,"['action', 'adventure', 'fantasy', 'sci-fi']",False,False,False,False,Who are you? / Where am I? / I can't see. / So...
591,m203,u3104,KAY,the godfather,f,377,1972,"['crime', 'drama', 'thriller']",False,True,True,False,Will you give this letter to Michael. / Will y...
837,m247,u3742,AGENT,apocalypse now,m,282,1979,"['drama', 'war']",False,False,True,False,I don't know what you're talking about -- Get ...


In [4]:
chars = chars.loc[~pd.isnull(chars['lines']), : ]
chars = chars.reset_index(drop = True)  # try commenting this line out, and see how it
                                          # makes your task more difficult
chars.shape

(2969, 13)

In [5]:
chars['lines'].value_counts()

Good.  That's very good. / Don't be.  I'm going.  Where we all go.  Now how can that be bad. / Are you afraid for me?  Where I'm going. / Keep Ben's for awhile, okay?  Until he's old enough to not open them all at once. / Let me be the judge of that. / Thank God.  I got to see you. Grown up. / Don't. Take me with you. / No, that never solves anything.  You talk like the big girl you are.  About what you feel.  And how he's hurt you. You know?  You tell the truth. / I know.  I know how rough life can be. And how unfair.  So here's what we do. Tomorrow, on the lunch yard, you walk <u>straight</u> <u>up</u> to that little jerk... / Darling, I've been thinking about our little Brad problem?  And I think it's time we move to Plan B... / Nothing sweetheart. / Annabelle...wake up honey... / How many devilled eggs can you eat? / Oh, yeh. / He's not even there.  You don't see him, you don't hear him, you're just too much of a woman to bother with little boys. / You have to ignore him. / And wha

In [7]:
vectorizer = CountVectorizer(max_features = 4000)
sparse_wordcounts = vectorizer.fit_transform(chars.lines)
wordcounts = sparse_wordcounts.toarray()
charwords = pd.DataFrame(wordcounts, columns = vectorizer.get_feature_names())
charwords.head()

,000,10,100,11,12,14,15,18,20,24,...,your,yours,yourself,yourselves,youth,yuh,yup,zero,zone,zoo
0,0,2,0,0,0,0,0,0,1,0,...,25,0,2,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,6,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,0,0,0


In [26]:
keepers = []
indeces = [] #zero = comedy, one = action movie
for idx, row in chars.iterrows():
    if 'action' in row.genres and 'comedy' in row.genres:
        continue
    if 'action' in row.genres or 'comedy' in row.genres:
        keepers.append(idx)
        if 'action' in row.genres:
            indeces.append(1)
        else:
            indeces.append(0)
            
#print(indeces)
sum_of_action = 0
sum_of_comedy = 0
for i in indeces:
    if i == 0:
        sum_of_comedy += 1
    else:
        sum_of_action += 1
        
print(sum_of_action)
print(sum_of_comedy)
for i in keepers:
    print(chars.loc[[i]])

542
756
    mid    cid   cname    mname gender  wordcount  year               genres  \
0  m533  u7883  JACKIE  stepmom      f       3037  1998  ['comedy', 'drama']   

   comedy  thriller  drama  romance  \
0    True     False   True    False   

                                               lines  
0  Good.  That's very good. / Don't be.  I'm goin...  
   mid    cid  cname       mname gender  wordcount  year  \
1  m70  u1070  BONES  get shorty      m        242  1995   

                            genres  comedy  thriller  drama  romance  \
1  ['comedy', 'crime', 'thriller']    True      True  False    False   

                                               lines  
1  You say look at you? / What? / Where's Leo Dev...  
    mid    cid cname                                       mname gender  \
2  m489  u7245   HAN  star wars: episode vi - return of the jedi      m   

   wordcount  year                                        genres  comedy  \
2        403  1983  ['action', 'adventu

166  When did you two talk this over? / Well -- whe...  
      mid    cid       cname         mname gender  wordcount  year  \
167  m186  u2828  CARRUTHERS  smokin' aces      m        281  2006   

                                       genres  comedy  thriller  drama  \
167  ['action', 'crime', 'drama', 'thriller']   False      True   True   

     romance                                              lines  
167    False  Yeah. / Mortal. / No.  I'm going over there. Y...  
     mid   cid   cname             mname gender  wordcount  year  \
170  m19  u320  PARKER  american outlaws      m        265  2001   

                    genres  comedy  thriller  drama  romance  \
170  ['action', 'western']   False     False  False    False   

                                                 lines  
170  There's only four of them... / How can that be...  
      mid    cid cname                         mname gender  wordcount  year  \
171  m217  u3283   TED  there's something about mary      m  

337  Aloha, Spicoli. / Don't worry, Spicoli. You'll...  
      mid    cid cname              mname gender  wordcount  year      genres  \
338  m605  u8893   DAD  who's your daddy?      m        317  2003  ['comedy']   

     comedy  thriller  drama  romance  \
338    True     False  False    False   

                                                 lines  
338  We think you should give up the magazine. Sell...  
      mid    cid cname     mname gender  wordcount  year               genres  \
339  m537  u7936  JEFF  suburbia      m       2571  1996  ['comedy', 'drama']   

     comedy  thriller  drama  romance  \
339    True     False   True    False   

                                                 lines  
339  You know, like, like, I would always think, uh...  
      mid    cid   cname                     mname gender  wordcount  year  \
340  m196  u2997  PICARD  star trek: first contact      m       1166  1996   

                                            genres  comedy  thrill

453  Yes, sir. / Uh, oh. / What are we going to do?...  
      mid    cid cname          mname gender  wordcount  year  \
455  m411  u6168   LEX  jurassic park      f        246  1993   

                                                genres  comedy  thriller  \
455  ['action', 'adventure', 'family', 'sci-fi', 'a...   False     False   

     drama  romance                                              lines  
455  False    False  Phone security systems, everything works.  You...  
      mid    cid     cname                     mname gender  wordcount  year  \
456  m452  u6774  JULIANNE  my best friend's wedding      f       2014  1997   

                    genres  comedy  thriller  drama  romance  \
456  ['comedy', 'romance']    True     False  False     True   

                                                 lines  
456  I'll think about it.  I'm okay. / I did what I...  
      mid    cid   cname                  mname gender  wordcount  year  \
457  m244  u3710  MONICA  the anni

624   True     True  I'm all right. / Hey, can we go to McDonald's?...  
    mid  cid   cname              mname gender  wordcount  year  \
628  m5  u85  LEELOO  the fifth element      f        233  1997   

                                                genres  comedy  thriller  \
628  ['action', 'adventure', 'romance', 'sci-fi', '...   False      True   

     drama  romance                                              lines  
628  False     True  Akta dedero ansila do mektet. / Vano da, mecht...  
      mid    cid cname        mname gender  wordcount  year  \
632  m266  u3998   EVE  being there      f       1371  1979   

                  genres  comedy  thriller  drama  romance  \
632  ['drama', 'comedy']    True     False   True    False   

                                                 lines  
632  I do - we do - both of us, Ben and I feel so m...  
      mid    cid  cname                  mname gender  wordcount  year  \
641  m244  u3703  CLAIR  the anniversary party      f

...

2128  Hank was always good with the ladies. Always g...  
      mid   cid cname            mname gender  wordcount  year  \
2133  m54  u839   AUD  erik the viking      f        385  1989   

                                  genres  comedy  thriller  drama  romance  \
2133  ['comedy', 'adventure', 'fantasy']    True     False  False    False   

                                                  lines  
2133  It's sinking! Hy-Brasil is sinking! / But, Fat...  
       mid    cid cname      mname gender  wordcount  year  \
2134  m142  u2202  BENI  the mummy      m        336  1999   

                                  genres  comedy  thriller  drama  romance  \
2134  ['action', 'adventure', 'fantasy']   False     False  False    False   

                                                  lines  
2134  Something about bringing his dead girly-friend...  
      mid   cid  cname            mname gender  wordcount  year  \
2137  m54  u846  HELGA  erik the viking      f        226  1989   

   

2287  WHEN SHE'S SAFE! / You do, and the spring thaw...  
       mid    cid   cname          mname gender  wordcount  year  \
2288  m246  u3732  MARGIE  the apartment      f        257  1960   

                              genres  comedy  thriller  drama  romance  \
2288  ['romance', 'comedy', 'drama']    True     False   True     True   

                                                  lines  
2288  You bet I will. And when I tell him how you tr...  
       mid    cid cname         mname gender  wordcount  year  \
2290  m462  u6906   MAX  notting hill      m        274  1999   

                     genres  comedy  thriller  drama  romance  \
2290  ['comedy', 'romance']    True     False  False     True   

                                                  lines  
2290  No! / Now you're lying. / I'm delighted. / I'm...  
       mid    cid  cname        mname gender  wordcount  year  \
2296  m598  u8810  BREAN  wag the dog      m       3555  1997   

                   genres  come

2436     True  Betty! Listen to me! Del is ... / Do what? / B...  
       mid    cid  cname      mname gender  wordcount  year  \
2441  m585  u8624  HELEN  true lies      f       1008  1994   

                      genres  comedy  thriller  drama  romance  \
2441  ['action', 'thriller']   False      True  False    False   

                                                  lines  
2441  Will you shutup.  I should never have told you...  
       mid    cid  cname             mname gender  wordcount  year  \
2444  m522  u7722  SUGAR  some like it hot      f       2567  1959   

          genres  comedy  thriller  drama  romance  \
2444  ['comedy']    True     False  False    False   

                                                  lines  
2444  He's going to South America to marry some othe...  
       mid    cid   cname  mname gender  wordcount  year               genres  \
2447  m519  u7686  VINNIE  smoke      m        215  1995  ['comedy', 'drama']   

      comedy  thriller  dram

2614   True    False  What are they doing? / It was in Latin, sir. /...  
      mid    cid  cname             mname gender  wordcount  year  \
2616  m97  u1440  MARTY  independence day      m        133  1996   

                                             genres  comedy  thriller  drama  \
2616  ['action', 'adventure', 'sci-fi', 'thriller']   False      True  False   

      romance                                              lines  
2616    False  Then what? / A countdown to what? / Not really...  
       mid    cid cname            mname gender  wordcount  year  \
2618  m126  u1928  LARA  minority report      f        325  2002   

                                                 genres  comedy  thriller  \
2618  ['action', 'crime', 'mystery', 'sci-fi', 'thri...   False      True   

      drama  romance                                              lines  
2618   True    False  John?  What is it? / I know... I do, too... / ...  
       mid    cid cname               mname gender  

2808  Now hold on, Baxter -- / What's gotten into yo...  
      mid    cid  cname       mname gender  wordcount  year  \
2809  m89  u1345  MORAN  highlander      m        528  1986   

                     genres  comedy  thriller  drama  romance  \
2809  ['action', 'fantasy']   False     False  False    False   

                                                  lines  
2809  Don't be stupid, lady.  Your neck can be slice...  
       mid    cid   cname   mname gender  wordcount  year  \
2810  m355  u5345  FLETCH  fletch      m       3180  1985   

                              genres  comedy  thriller  drama  romance  \
2810  ['comedy', 'crime', 'mystery']    True     False  False    False   

                                                  lines  
2810  Comanche Indian. / God I admire you. / I feel ...  
       mid    cid       cname      mname gender  wordcount  year  \
2811  m579  u8540  MRS. DAVIS  toy story      f        177  1995   

                                           

In [ ]:
title_basics = pd.read_csv('/Users/michelleschlacks/Desktop/final project IS417/title.basics.tsv')
title_ratings = pd.read_csv('/Users/michelleschlacks/Desktop/final project IS417/title.ratings.tsv')

In [ ]:
title_basics